<a id='top-page'></a>
# <img src="../images/PCAfold-logo.svg" style="height:100px"> Handling source terms

This tutorial can be of interest to researchers working with reactive flows data sets. We present how source terms of the original state variables can be handled using **PCAfold** software. Specifically, **PCAfold** functionalities accommodate treatment of sources of principal components (PCs) which can be valuable for implementing PC-transport approaches such as proposed in [[1]](#references).

- [**Theory**](#theory)
- [**Code implemention**](#implementation)
- [**References**](#references)

<a id='theory'></a>

***

## Theory

[**↑ Go to the top**](#top-page)

The methodology for the standard PC-transport approach was first proposed
in [[1]](#references). As an illustrative example, PC-transport
equations adequate to a 0D chemical reactor are presented below.
The reader is referred to [[2]](#references), [[3]](#references) for treatment of the full PC-transport equations including diffusion.

We assume that the data set containing original state-space variables is:

$$
  \mathbf{X} = [T, Y_1, Y_2, \dots, Y_{N_s-1}]
$$

where $T$ is temperature and $Y_i$ is a mass fraction of species
$i$. $N_s$ is the total number of chemical species. The corresponding
source terms of the original state-space variables are:

$$
  \mathbf{S_X} = \Big[ -\frac{1}{\rho c_p} \sum_{i=1}^{N_s} ( \omega_i h_i ), \frac{\omega_1}{\rho}, \frac{\omega_2}{\rho}, \dots, \frac{\omega_{N_s-1}}{\rho} \Big]
$$

where $\rho$ is the density of the mixture and $c_p$ is the specific heat capacity of the mixture, $\omega_i$ is the net mass production rate of species $i$ and $h_i$ is the enthalpy of species $i$.

For a 0D-system, we can write the evolution equation as:

$$
  \frac{d \mathbf{X}}{dt} = \mathbf{S_X}
$$

This equation can be instead written in the space of principal components by applying
a linear operator $\mathbf{A}$ identified by PCA. We can also account for
centering and scaling the original data set $\mathbf{X}$ using centers
$\mathbf{C}$ and scales $\mathbf{D}$:

$$
  \frac{d \Big( \frac{\mathbf{X} - \mathbf{C}}{\mathbf{D}} \Big) \mathbf{A}}{dt} = \frac{\mathbf{S_X}}{\mathbf{D}}\mathbf{A}
$$

It is worth noting that when the original data set is centered and scaled,
the corresponding source terms should only be scaled and not centered, since:

$$
  \frac{d \frac{\mathbf{C}}{\mathbf{D}} \mathbf{A}}{dt} = 0
$$

for constant $\mathbf{C}$, $\mathbf{D}$ and $\mathbf{A}$.

We finally obtain the 0D PC-transport equation where the evolved variables
are principal components instead of the original state-space variables:

$$
  \frac{d \mathbf{Z}}{dt} = \mathbf{S_{Z}}
$$

where $\mathbf{Z} = \Big( \frac{\mathbf{X} - \mathbf{C}}{\mathbf{D}} \Big) \mathbf{A}$
and $\mathbf{S_{Z}} = \frac{\mathbf{S_X}}{\mathbf{D}}\mathbf{A}$.

<a id='implementation'></a>
***

## Code implementation

[**↑ Go to the top**](#top-page)

We import the necessary modules:

In [ ]:
from PCAfold import PCA
import numpy as np

A data set representing combustion of syngas in air generated from steady laminar flamelet model using [*Spitfire*](https://github.com/sandialabs/Spitfire) software [[4]](#references) and a chemical mechanism by Hawkes et al. [[5]](#references) is used as a demo data set. 

We begin by importing the data set composed of the original state space variables
$\mathbf{X}$ and the corresponding source terms $\mathbf{S_X}$:

In [2]:
X = np.genfromtxt('data-state-space.csv', delimiter=',')
S_X = np.genfromtxt('data-state-space-sources.csv', delimiter=',')

We perform PCA on the original data:

In [3]:
pca_X = PCA(X, scaling='auto', n_components=2)

We transform the original data set  to the newly identified basis and
compute the principal components (PCs) $\mathbf{Z}$:

In [4]:
Z = pca_X.transform(X, nocenter=False)

Transform the source terms to the newly identified basis and compute the sources
of principal components $\mathbf{S_Z}$:

In [5]:
S_Z = pca_X.transform(S_X, nocenter=True)

Note that we set the flag ``nocenter=True`` which is a specific setting that
should be applied when transforming source terms.
With that setting, only scales $\mathbf{D}$ will be applied when transforming $\mathbf{S_X}$
to the new basis defined by $\mathbf{A}$ and thus the transformation will be consistent with the discussion presented
in the previous section.

<a id='references'></a>
***

## References

[**↑ Go to the top**](#top-page)

> [[1]](#theory) James C. Sutherland and Alessandro Parente, *Combustion modeling using principal component analysis*, Proceedings of the Combustion Institute, 32(1):1563–1570, 2009.
>
> [[2]](#theory) A. Biglari and J. C. Sutherland, *An a-posteriori evaluation of principal component analysis-based models for turbulent combustion simulations*, Combustion and Flame, 162(10):4025–4035, 2015.
> 
> [[3]](#theory) T. Echekki, H. Mirgolbabaei, *Principal component transport in turbulent combustion: a posteriori analysis*, Combustion and Flame, 162(5):1919–1933, 2015.
>
> [[4]](#implementation) [M. A. Hansen, *Spitfire*, 2020](https://github.com/sandialabs/Spitfire)
> 
> [[5]](#implementation) E. R. Hawkes, R. Sankaran, J. C. Sutherland, J. H. Chen, *Scalar mixing in direct numerical simulations of temporally evolving plane jet flames with skeletal co/h2 kinetics*, Proceedings of the combustion institute 31 (1) (2007) 1633–1640

***